<div style="background:#5D6D7E;padding:20px;color:#ffffff;margin-top:10px;">

# NLP - Práctica 1 ( Extracción de Información) 

## Profesora: Lisibonny Beato
### Período 3-2024-2025</div>


In [166]:
# Importando algunas librerias

import pandas as pd
import re
import spacy
import textacy
import glob
import visualise_spacy_tree
from spacy.matcher import Matcher 
import os
import sys
import en_core_web_trf
import spacy_transformers
import re
import nltk
from collections import Counter


#añadidas por el grupo
import requests
from sklearn.feature_extraction.text import TfidfVectorizer

In [167]:
#pip install pandas

In [168]:
#pip install spacy

In [169]:
#pip install textacy

In [170]:
#pip install visualise-spacy-tree

In [171]:
#pip install en-core-web-trf

In [172]:
#pip install spacy-transformers

In [173]:
#pip install nltk

In [174]:
#pip install --upgrade numpy

In [175]:
#!python -m spacy download en_core_web_sm


In [176]:
#pip install --upgrade h5py

In [177]:
#!python -m spacy download en_core_web_trf

<div style="background:#ff6242;padding:20px;color:#ffffff;margin-top:10px;">
<b>Para esta práctica estarán utilizando el corpus MIT Restaurant Corpus, el cual encontrará en este enlace: <a href='https://sls.csail.mit.edu/downloads/restaurant/'>https://sls.csail.mit.edu/downloads/restaurant/</a>.<br /><br />Tome en cuenta que los archivos de train y test están en el formato bio.
<br />
<br />
Aparte del código, debe proveer una interpretación para cada tarea y un análisis para cada resultado obtenido que así lo amerite.</b>
</div>


In [178]:

# # Descargar archivo de entrenamiento
# url_train = "https://sls.csail.mit.edu/downloads/restaurant/restauranttrain.bio"
# response_train = requests.get(url_train)
# with open("restauranttrain.bio", "wb") as f:
#     f.write(response_train.content)

# # Descargar archivo de prueba
# url_test = "https://sls.csail.mit.edu/downloads/restaurant/restauranttest.bio"
# response_test = requests.get(url_test)
# with open("restauranttest.bio", "wb") as f:
#     f.write(response_test.content)


## 1. Nivel Básico 
### Puntuación máxima de la tarea: 5 puntos
#### Limpieza y preparación de los datos, Extracción de palabras clave (KPE),  y reconocimiento de entidades relevantes (NER) con modelos heurísticos y pre-entrenados para  reconocer al menos 3 tipos de entidades.

### Extraccion y limpiueza de los datos

In [179]:
def load_bio_data(filepath):
    sentences = []
    sentence = []
    with open(filepath, 'r', encoding="utf-8") as file:
        for line in file:
            if line.strip() == "":
                if sentence:
                    sentences.append(sentence)
                    sentence = []
            else:
                parts = line.strip().split()
                if len(parts) >= 2:
                    tag, word = parts[0], " ".join(parts[1:])  # ← invertido aquí
                    sentence.append((word, tag))
    return sentences


In [180]:
sentences = load_bio_data('restauranttrain.bio')
#test_data = load_bio_data("restauranttest.bio")

In [181]:
# Verificar el contenido real del archivo .bio
with open("restauranttrain.bio", "r", encoding="utf-8") as f:
    for i in range(30):
        print(f.readline().strip())


B-Rating	2
I-Rating	start
O	restaurants
O	with
B-Amenity	inside
I-Amenity	dining

O	34

B-Rating	5
I-Rating	star
O	resturants
B-Location	in
I-Location	my
I-Location	town

O	98
B-Restaurant_Name	hong
I-Restaurant_Name	kong
O	restaurant
B-Price	reasonable
O	prices

O	a
O	great
O	lunch
O	spot
O	but
B-Hours	open
I-Hours	till


In [182]:
documentos = [" ".join([word for word, tag in sent]) for sent in sentences[:100]]
vectorizer = TfidfVectorizer(stop_words="english", max_features=20)
tfidf_matrix = vectorizer.fit_transform(documentos)


In [183]:
top_keywords = vectorizer.get_feature_names_out()
df_keywords = pd.DataFrame({"Palabras Clave (TF-IDF)": top_keywords})
print(df_keywords)

   Palabras Clave (TF-IDF)
0                    cheap
1                   coffee
2                  cuisine
3                   dining
4                      eat
5                     fast
6                     food
7                     good
8                    great
9                    miles
10                  nearby
11                    open
12                   place
13                  places
14              restaurant
15             restaurants
16                   serve
17                  serves
18                   steak
19                    want


In [184]:
entidades = []
actual_entidad = ""
tipo_actual = ""

for sent in sentences:
    for word, tag in sent:
        if tag.startswith("B-"):
            if actual_entidad:
                entidades.append((actual_entidad.strip(), tipo_actual))
            actual_entidad = word + " "
            tipo_actual = tag[2:]
        elif tag.startswith("I-") and tipo_actual == tag[2:]:
            actual_entidad += word + " "
        else:
            if actual_entidad:
                entidades.append((actual_entidad.strip(), tipo_actual))
                actual_entidad = ""
                tipo_actual = ""

if actual_entidad:
    entidades.append((actual_entidad.strip(), tipo_actual))

import pandas as pd
df_entidades_custom = pd.DataFrame(entidades, columns=["Texto", "Etiqueta"])
print(df_entidades_custom.head(10))


                  Texto         Etiqueta
0               2 start           Rating
1         inside dining          Amenity
2                5 star           Rating
3            in my town         Location
4             hong kong  Restaurant_Name
5            reasonable            Price
6       open till 2 a m            Hours
7       passims kitchen  Restaurant_Name
8  soft serve ice cream             Dish
9                  good           Rating


<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Utilice esta celda para colocar comentarios en el notebook, cuando lo estime necesario. Copiela varias veces donde considere.
</div>

## 2. Nivel Intermedio 
### Puntuación máxima de la tarea: 2 puntos
#### Entrenar un modelo propio de NER mediante el pipeline de Spacy

## 3. Nivel Avanzado
### Puntuación máxima de la tarea: 3 puntos
#### Entrenar un modelo de NER mediante el uso de Conditional Random Fields. Recuerde ir más allá que donde llega el ejemplo al respecto provisto en clases.

In [ ]:
#pip install sklearn-crfsuite


Note: you may need to restart the kernel to use updated packages.


In [186]:
def word2features(sent, i):
    word = sent[i][0]
    features = {
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True  # Beginning of sentence

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True  # End of sentence

    return features

def extract_features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def get_labels(sent):
    return [label for token, label in sent]


In [187]:

X = [extract_features(s) for s in sentences]
y = [get_labels(s) for s in sentences]


In [188]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [189]:
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,  
    c2=0.1,  
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(X_train, y_train)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    max_iterations=100)

In [190]:
from sklearn_crfsuite import metrics

y_pred = crf.predict(X_test)

labels = list(crf.classes_)
labels.remove('O')  # No evaluar "Outside"
f1 = metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)
print("F1-score (sin 'O'):", f1)

F1-score (sin 'O'): 0.7972937389714859


In [191]:
report = metrics.flat_classification_report(y_test, y_pred, labels=labels, digits=3)
print(report)

                   precision    recall  f1-score   support

        B-Amenity      0.739     0.690     0.714       500
        B-Cuisine      0.836     0.843     0.839       579
       B-Location      0.874     0.852     0.863       764
B-Restaurant_Name      0.863     0.821     0.842       369
I-Restaurant_Name      0.763     0.775     0.769       311
       I-Location      0.829     0.879     0.853       705
           B-Dish      0.789     0.723     0.755       311
          B-Hours      0.762     0.730     0.746       215
          I-Hours      0.875     0.879     0.877       280
        I-Amenity      0.722     0.724     0.723       525
         B-Rating      0.819     0.797     0.808       222
         I-Rating      0.788     0.713     0.749       115
        I-Cuisine      0.690     0.617     0.652       141
           I-Dish      0.707     0.656     0.680       151
          B-Price      0.887     0.815     0.849       135
          I-Price      0.778     0.660     0.714       

In [192]:
import joblib
joblib.dump(crf, "modelo_ner_crf.pkl")


['modelo_ner_crf.pkl']